Abstract and explination here

In [13]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import threading
import time

## Group Assignment
### Team Number: 05
### Team Member Names: Piero Camposeo, Sathun Suthakaran, Ishaan Bansal
### Team Strategy Chosen: SAFE

In [14]:
#Variables used to call on the Queue
global queueC
global queueV

#Queue that is used for the purpose of retrieving variables from threads
class Queue(object):
    def __init__(self):
        self.item = []
        
    def __repr__(self):
        return "{}".format(self.item)
    
    def __str__(self):
        return "{}".format(self.item)
    
    def enque(self, add):
        self.item.insert(0, add)
        return True
    
    def size(self):
        return len(self.item)
    
    def isempty(self):
        if self.size() == 0:
            return True
        else:
            return False
        
    def deque(self):
        if self.size() == 0:
            return None
        else:
            return self.item.pop()

In [15]:
#Read in csv file of tickers
csv_file = "Tickers.csv"

# Make values into a list
tickers = pd.read_csv(csv_file)
ticker_list = tickers.iloc[:, 0].values.tolist()
ticker_list.insert(0, tickers.columns[0])

In [16]:
#Consumes desired amount of trading days within a month, a dataframe of days, a dstaframe of months, and a list of months that cooresponds to the months
#    on the dataframe, outputs the monthly dataframe with only months that have 'daycount' amount of trading days for that stock
#Note: Function will cover up to the last date. For example, the function will not produce October for a dataframe which ends on October 1st.
#   In our code, we simply passed a days dataframe that ends one month after the months dataframe does (*)
def df_with_valid_months(daycount, df_days, df_months, months):
    for month in range (len(months)):
        if len(df_days.filter(like = months[month], axis = 0)) < daycount:
            df_months.drop(df_months.index[month], axis = 0, inplace = True) #Drop any months with not enough trading days
            
    return(df_months)

#Consumes historical data for a stock, outputs average volume
def average_volume(df_hist):
    volume = df_hist['Volume'].dropna().values.tolist()
    vol_av = np.mean(volume)

    return(vol_av)


In [17]:
#dates for volume calculation, end date for valid months function (see (*))
vol_date_s = '2022-01-01'
vol_date_e = '2022-10-02'
endDForValidMonthsFunct = str((pd.to_datetime(vol_date_e) + relativedelta(months = 1)).strftime("%Y-%m-%d"))

#for df_with_valid_months function, creates a series of months from vol_date_s to vol_date_e
months = pd.date_range(vol_date_s, vol_date_e, freq='MS').strftime("%Y-%m").tolist()

#As outlined in assignment information
minimumMonthlyTradingDays = 20
minimum_volume = 200000
required_denomination = 'USD'

#Here we will put valid stocks that fit our criteria
tickers = []

#Calling on Queue class
queueC = Queue()
queueV = Queue()

#Consumes a list of tickers, checks if they are denominated in 'required_denomination'.
def currency_check(ticker_list):
    valid = []
    
    #yf.Tickers object of all tickers
    all_symbols  = " ".join(ticker_list)
    tickers = yf.Tickers(all_symbols)

    #Retrieve financial currency from each ticker, if it is 'required_denomination', it is a valid ticker. Else, we don't care so we set it as nan
    for ticker in ticker_list:
        try:
            currency = tickers.tickers[ticker].info['financialCurrency']
        except:
            currency = 'nan'
        
        if currency == required_denomination:
            valid.append(ticker)
    
    #Pass our valid tickers to the QueueC (Queue Currency), so that they may be called on later
    queueC.enque(valid)
        
#Consumes a list of tickers, checks if they have an average monthly volume of 'minimum_volume' or more over each month with 
# 'minimumMonthlyTradingDays' or more trading days
def volume_check(ticker_list):
    valid = []
    
    for ticker in ticker_list:
        tick = yf.Ticker(ticker)
        
        #Check for Delisted ticker + Creating histories. If 'check' returns an error, the code continues. Without this try/except,
        #Error code for delisted stock appears twice
        try:
            tick_hist = tick.history(start = vol_date_s, end  = vol_date_e, interval = '1mo').dropna()
            check = tick_hist.iloc[0,0]
            #Custom dataframe for valid months check (daily, ending one month after monthly dataframe)
            tickHistForValidMonthsCheck = tick.history(start = vol_date_s, end  = endDForValidMonthsFunct, interval = '1d').dropna()
 
            #Dataframe with valid months only
            tickHistWithValidMonths = df_with_valid_months(minimumMonthlyTradingDays, tickHistForValidMonthsCheck, tick_hist, months)
       
        except:
            pass
            
        #Call on average_volume function to compute and average volume
        monthly_average_volume = 0 #Initialize this to 0 to prevent errors
        monthly_average_volume = average_volume(tickHistWithValidMonths)
        
        #Check, append to valid if criteria is fit
        if monthly_average_volume >= minimum_volume:
            valid.append(ticker)

    #Pass our valid tickers to the QueueV (Queue Volume), so that they may be called on later
    queueV.enque(valid)
        
#Creating threads for currency_check and volume_check function
c = threading.Thread(target = currency_check, args = (ticker_list,))
v = threading.Thread(target = volume_check, args = (ticker_list,))

#Start thread executions
c.start()
v.start()

#Wait for threads to finish and retrieve valid tickers from respective queues
c.join()
currency_valid = queueC.deque()
v.join()
volume_valid = queueV.deque()

#Valid tickers are the intersection of the two lists (Must have particular denomination and average monthly volume as outlined in assignment)
tickers = [value for value in currency_valid if value in volume_valid]

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted


In [ ]:
#Sathun

In [ ]:
#Ishaan FOR LATER: WRITE WHY VOLATILITY IS WITHOUT TIME PERIOD IN EXPLANATION SOMEWHERE (LOOK AT VOLATILITY FORMULA)
# Consumes a Dataframe of a list of stocks and their daily percent returns and produces a list of the 25 least volatile stocks
def lowest_volatile_stocks(pr_df):
    
    all_stock_vol = []

    # Performs the operations for every column which is a stock
    for i in range(len(pr_df.columns)):
        # Calculates the standard deviation of the stock's daily percent return
        stock_vol = pr_df.iloc[:,i].std()
        
        all_stock_vol.append(stock_vol)

    # Creates a dataframe that will store each stock and their volatility and sort by non-decreasing volatility
    all_stock_vol_df = pd.DataFrame({"Stocks": list(pr_df.columns), "Volatility": all_stock_vol,})
    all_stock_vol_df.sort_values(by = "Volatility", inplace = True)

    # Creates a list of the least volatile stocks of size 25 or the number of stocks in the dataframe (whichever is the minimum value)
    portfolio = list(all_stock_vol_df['Stocks'].head(min(len(pr_df.columns)), 25))
    
    return portfolio

In [ ]:
# Consumes a list of tickers and a DataFrame. The requirements of the DataFrame are:
#   a) df is either a DataFrame of closing prices of stocks with the stock name as the column header or a DataFrame of percent returns of stocks 
#      with the stock name as the column header
#   b) df must include all of the tickers in list_of_ticks
# The function returns a dataframe of the stocks from list_of_ticks and either their closing prices or daily percent returns, depending on what 
# was inputted, with the stock name as the column header
def lowest_volatile_stocks_df_filter(list_of_ticks, df):
    # Creates the dataframe to store the data and creates the first column with the first stock's data from list_of_ticks
    final_df = pd.DataFrame({list_of_ticks[0]: df[list_of_ticks[0]],})

    # Creates a new column with the stock's data and the stock name as the header for every ticker in list_of_ticks, excluding the first ticker
    for i in range(1,25):
        final_df[list_of_ticks[i]] = df[list_of_ticks[i]]

    return final_df

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CSCO',
 'CVS',
 'GM',
 'GOOG',
 'JPM',
 'KMI',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NKE',
 'ORCL',
 'OXY',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'SBUX',
 'SLB',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Piero Camposeo, Sathun Suthakaran, Ishaan Bansal